In [0]:
from __future__ import absolute_import, division, print_function

!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print (device_name)

    100% |████████████████████████████████| 332.1MB 62kB/s 
    100% |████████████████████████████████| 61kB 22.8MB/s 
    100% |████████████████████████████████| 3.0MB 8.1MB/s 
    100% |████████████████████████████████| 419kB 11.9MB/s 
/device:GPU:0


In [0]:
import matplotlib.pyplot as plt
% matplotlib inline
import numpy as np
import pandas as pd
import time
import random

In [0]:
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
batch_size = 32
emb_size = 50
channel = 50
kernel_size = 3
lr = 1e-3

In [0]:
train_path = '/content/drive/My Drive/Colab Notebooks/cs11747/data/class/train.txt' # train set
dev_path = '/content/drive/My Drive/Colab Notebooks/cs11747/data/class/dev.txt' # dev set

In [0]:
def read_dataset(filename):
    data = []
    label = []
    with open(filename, "r") as f:
        for line in f:
            tag, words = line.lower().strip().split(" ||| ")
            data.append(words)
            label.append(tag)
        return pd.DataFrame({'data': data, 'label': label})

In [0]:
train_set = read_dataset(train_path).sample(frac=1)
dev_set = read_dataset(dev_path)
print (train_set.shape, dev_set.shape)

(8544, 2) (1101, 2)


In [0]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<unk>')
tokenizer.fit_on_texts(pd.concat([train_set['data'], dev_set['data']]))

In [0]:
word_index = tokenizer.word_index
nwords = len(word_index)
print (nwords)

16219


In [0]:
def train_gen(batch_size=batch_size):
    steps = train_set.shape[0] // batch_size
    for step in range(steps):
        train_batch = train_set.iloc[step:step+batch_size]
        train_seq = tokenizer.texts_to_sequences(train_batch['data'])
        maxlen = max(map(len, train_seq))
        train_pad = tf.keras.preprocessing.sequence.pad_sequences(train_seq, maxlen=maxlen)
        train_label = list(map(int, train_batch['label']))
        yield train_pad, train_label

In [0]:
def dev_gen(batch_size=batch_size):
    steps = dev_set.shape[0] // batch_size
    for step in range(steps):
        dev_batch = dev_set.iloc[step:step+batch_size]
        dev_seq = tokenizer.texts_to_sequences(dev_batch['data'])
        maxlen = max(map(len, dev_seq))
        dev_pad = tf.keras.preprocessing.sequence.pad_sequences(dev_seq, maxlen=maxlen)
        dev_label = list(map(int, dev_batch['label']))
        yield dev_pad, dev_label

In [0]:
class CNNclass(tf.keras.Model):
    def __init__(self, nwords, emb_size, channel, kernel_size, ntags):
        super(CNNclass, self).__init__()
        self.embed = tf.keras.layers.Embedding(nwords+1, emb_size, trainable=True)
        self.conv = tf.keras.layers.Conv1D(filters=channel, kernel_size=kernel_size, strides=1, padding='valid', dilation_rate=1, activation='relu')
        self.gmp = tf.keras.layers.GlobalMaxPool1D()
        self.dense = tf.keras.layers.Dense(ntags, activation='softmax')

    @tf.function
    def call(self, x):
        """Run the model."""
        result = self.embed(x)
        result = self.conv(result)
        result = self.gmp(result)
        result = self.dense(result)

        return result

In [0]:
model = CNNclass(nwords, emb_size, channel, kernel_size, ntags=5)

In [0]:
# Calculate the loss value for the whole batch of sentences
def sent_loss(sents, labels):
    all_sents = tf.Variable(sents, trainable=False)
    all_targets = tf.Variable(labels, trainable=False)
    logits = model(all_sents)
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.losses.Reduction.SUM)
    loss = loss_fn(all_targets, logits)

    return loss

In [0]:
last_dev = 1e20
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# Iterate over epochs.
for epoch in range(5):
    print('Start of epoch %d' % (epoch,))
    start = time.time()
    train_loss = 0
    batch_id = 0
    for sents, labels in train_gen():

        # Open a GradientTape to record the operations run during the forward pass, which enables autodifferentiation.
        with tf.GradientTape() as tape:

            loss = sent_loss(sents, labels)
            train_loss += loss

            # Use the gradient tape to automatically retrieve the gradients of the trainable weights with respect to the loss.
            grads = tape.gradient(loss, model.trainable_variables)

            # Run one step of gradient descent by updating the value of the weights to minimize the loss.
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

            # Log every 50 batch.
            batch_id += 1
            if batch_id % 50 == 0:
                print('Training time: %0.3f seconds, training loss at sentence %d: %0.4f' % (time.time()-start, batch_id*batch_size, loss/batch_size))
                
    print ('Epoch %d: Training time=%0.3f seconds, training loss per word=%0.4f' % (epoch, time.time()-start, train_loss/train_set.shape[0]))
    
    # Evaluate on dev set
    dev_loss = 0
    for sents, labels in dev_gen():
        loss = sent_loss(sents, labels)
        dev_loss += loss
    print ('Epoch %d: evaluation loss per word=%f' % (epoch, dev_loss/dev_set.shape[0]))
    
    # Keep track of the development accuracy and reduce the learning rate if it got worse
    if last_dev < dev_loss/dev_set.shape[0]:
        lr /= 2
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        print ('learning rate decay to: ', lr)
    last_dev = dev_loss/dev_set.shape[0]

Start of epoch 0
Training time: 15.308 seconds, training loss at sentence 1600: 1.3064
Training time: 29.564 seconds, training loss at sentence 3200: 0.9820
Training time: 43.780 seconds, training loss at sentence 4800: 0.6520
Training time: 58.128 seconds, training loss at sentence 6400: 0.5953
Training time: 72.723 seconds, training loss at sentence 8000: 0.4092
Epoch 0: Training time=77.508 seconds, training loss per word=0.8900
Epoch 0: evaluation loss per word=2.309846
Start of epoch 1
Training time: 14.601 seconds, training loss at sentence 1600: 0.0969
Training time: 29.317 seconds, training loss at sentence 3200: 0.0653
Training time: 43.524 seconds, training loss at sentence 4800: 0.0222
Training time: 58.479 seconds, training loss at sentence 6400: 0.0181
Training time: 73.169 seconds, training loss at sentence 8000: 0.0447
Epoch 1: Training time=78.153 seconds, training loss per word=0.0893
Epoch 1: evaluation loss per word=1.968752
Start of epoch 2
Training time: 14.918 sec

KeyboardInterrupt: ignored